# LROSE Introductory SAMURAI Tutorial

---

This interactive tutorial takes you through the most basic steps of how to run SAMURAI. SAMURAI is a variational analysis technique that is described in Bell et al. (2012), Foerster et al. (2014), Foerster and Bell (2017), Cha and Bell (2021), and other publications. The SAMURAI analysis yields a maximum likelihood estimate of the atmospheric state for a given set of observations and error estimates by minimizing a variational cost function. It has more features and more development than FRACTL, and is generally recommended over the former for publication-quality analysis. The two programs together provide a powerful combination to produce high quality multi-Doppler wind fields from ground-based, airborne, or mixed configurations.

---


## Tutorial Overview

### 1. Setup 

<!---
#### Directory organization 

<div>
<img src="../images/wind_guided_structure.png" width="500"/>
</div>

-->

#### QC-ed input data, a parameter file, and a center file:

**a. QC-ed radar data files are provided:**
* cfrad.20090605_220449.252_to_20090605_220456.362_DOW6_PPI.nc
* cfrad.20090605_220450.046_to_20090605_220457.156_DOW7_PPI.nc

*The QC process is not included in this tutorial. An example of one type of QC can be found in the QC tutorial and HawkEdit/solo are another tool to use for QC-ing data.

### 2. Basic FRACTL and SAMURAI "how to"

SAMURAI runs on the command line, using simple commands like this:

* <code lang="bash">samurai -params /path/to/samurai.params</code>

All of the information about how SAMURAI should analyze data (e.g., paths to input data, variable names, grids, filters) is defined in parameter files for each application. Therefore, most of the work to run SAMURAI goes into QC-ing the data, setting up the parameter files, and placing files in the correct location.

SAMURAI requirements:
* data (e.g., radar data, radiosondes, in-situ aircraft data)
* center file
* parameter file
* optional: background file

#### How to set up SAMURAI

**a. Set up directory structure:**

All data and the center file defining the Cartesian grid that SAMURAI will use need to be in the same directory. This tutorial will set up input and output directories for the initial SAMURAI exercises. *For additional SAMURAI runs, we recommend creating new input and output directories to keep all analyses separate.*

**b. Center file:**

The .cen file will be generated by the user through the provided Perl script. SAMURAI uses the center file to define the Cartesian frame of reference for the analysis, which be either be static or moving. It can be used to time-space correct the position of radar data in a moving storm system. It also sets the temporal limits for data to be included in the analysis. The center files are named according to the following convention: YYYYMMDD.cen
* Provided: Generate_center.pl

**c. Parameter files:**
SAMURAI's parameter file contains information about data I/O, the cartesian grid, variable names, filters, and other scientific choices. In this tutorial, the user will create their own parameter file from scratch and fill in the important parameters.


#### Environment and packages¶

First, we import the required python packages to run this notebook. Most of the LROSE processing can be done with the os package and shell commands. 

In [ ]:
import os
#### Need to modify later
os.environ['BASE_DIR'] = '/home/jovyan/lrose-hub'
os.environ['RAW_DIR'] = '/home/jovyan/share/raw'
base_dir = os.environ['BASE_DIR']
!echo "Base directory: "$BASE_DIR

We will copy the required data to one directory as SAMURAI requires and organize the directory by performing the following commands:

In [ ]:
# make subdirectory within data for input and output directory
!rm -rf ${BASE_DIR}/data/intro_samurai/input
!mkdir -p ${BASE_DIR}/data/intro_samurai/input

# create samurai output directory
!rm -rf ${BASE_DIR}/data/intro_samurai/output
!mkdir -p ${BASE_DIR}/data/intro_samurai/output

# copy raw files to input directory
!cp ${RAW_DIR}/wind_guided/*.nc ${BASE_DIR}/data/intro_samurai/input
!cp ${RAW_DIR}/wind_guided/Generate_center.pl ${BASE_DIR}/params/intro_samurai


### 2. Prepare data for analysis

In this tutorial, the provided data have already been QC-ed. For a standard procedure, users are recommended to perform the following procedure:

* Convert radar data from level 2 or other native format to CfRadial using RadxConvert 
    - SAMURAI can also incorporate non-radar data, and can read many commonly used formats. All data files in the <code lang="bash">input</code> directory will be included in the analysis. The read subroutine for each datatype is determined by the file suffix.

* Quality-controlled raw data by desired QC-tools. See the QC tutorial for an example of how QC can be done; HawkEdit and Solo can also be used to QC data.

Now that we have generated the parameter file, we can inspect the file. In JupyterHub, you can edit the file either in a terminal with vi or by opening the text file via the lefthand menu - whichever you prefer! The file should be located here: /home/jovyan/lrose-hub/params/wind_guided

<div class="alert alert-block alert-info"> <b>Task: Open the parameter file for modification. Feel free to open the text file using the navigation menu to the left or use your favorite command line text editor (e.g., vi).</b> 
</div>

#### 3.2 Modify the parameter file

Now we will step through and edit the parameter file.

First, we'll start with the parameters associated with the input/output data directories and filenames.

| Parameter | Line # | Description |
| --- | --- | --- |
|inDir| 391 |The input data directory. It can be a relative path, but an absolute path is more certain.|
|fileRegex| 399 |A regular expression to identify the files used for analysis. Usually ^cfrad, to find CfRadial files.|
|outNc| 426 |The output directory where the analysis will go.|
|outTxt| 417 |the name of the text file that will contain the verification of grid results.|

<div class="alert alert-block alert-info"> <b>Task: change the inDir, fileRegex, and outNc parameters.</b> 
    <br>
    Refer to the table above to find the parameters in the text file.

1. In the parameter file, change <code lang="bash">inDir = "/home/jovyan/lrose-hub/data/wind_guided/input"</code>

2. In the parameter file, change <code lang="bash">fileRegex = "^cfrad"</code>

3. In the parameter file, change <code lang="bash">outNc = "/home/jovyan/lrose-hub/data/wind_guided/output_frac/"</code>

4. In the parameter file, change <code lang="bash">outTxt = "/home/jovyan/lrose-hub/data/wind_guided/fractl_output.txt"</code>

</div>


---

Next, we'll define the cartesian grid.

| Parameter | Line # | Description |
| --- | --- | --- |
|zGrid, yGrid, xGrid| 174, 182, 190 |These determine the grid spacing and size of the domain. Provide either the grid spacing to determine the max and min from the data or provide "min,max,incr" for each parameter.|
|projLat0, projLon0| 229, 237 |The origin is an arbitrarily chosen point, but should be relevant for your objective. For example, it can be the geographical center of your multi-radar domain, the physical location of a radar for a single-radar domain, or the location of a feature of interest within your dataset (such as storm center). The latitude and longitude of your chosen origin should be given in decimal degrees.|
|radarAlt| 245 |Altitude of the radar (km). Sometimes mobile radars have the altitude entered in m, which can often cause errors in FRACTL.|

<div class="alert alert-block alert-info"> <b>Task: set up the FRACTL grid using the following values.</b> 
    <br>
    We have provided the grid information for you. In this example, we will make a small box around the Goshen tornado with 100 m grid spacing. Since the radar had limited elevation angles, we restrict the vertical grid to 5-km altitude. Normally, you want to make the domain taller than your data.

1. In the parameter file, change <code lang="bash">zGrid = "0,5,0.5"</code>

2. In the parameter file, change <code lang="bash">yGrid = "-5,15,0.1"</code>

3. In the parameter file, change <code lang="bash">xGrid = "-20,-1,0.1"</code>

4. In the parameter file, change <code lang="bash">projLat0 = 41.6144</code>

5. In the parameter file, change <code lang="bash">projLon0 = -104.252</code>

6. In the parameter file, change <code lang="bash">radarAlt = 1.488</code>

</div>


---

Finally, we'll define the variable names so that FRACTL knows which variables to use in the analysis.

| Parameter | Line # | Description |
| --- | --- | --- |
|radialName| 440 |Variable name for the Doppler velocity.|
|dbzName| 448 |Variable name for the radar reflectivity.|
|ncpName| 456 |Variable name for the normalized coherent power. This variable can be used to do some simple QC thresholding. If you don't have NCP or SQI in your data then just point this to any other variable and select a value above which all data will be included.|
|minDbz| 58 |Any values below the minimum reflectivity value will be tossed out. Example: -20.|
|minNcp| 69 |Any values below the minimum NCP will be tossed out. Some radars do not have NCP, so the NCP variable can be replaced by any other variable to perform a simple quality control (e.g., SW < -1, dBZ < -20). This is generally intended for quick realtime quality control process.|

<div class="alert alert-block alert-info"> <b>Task: look for the variable names for reflectivity, velocity, and whether NCP exists in each file.</b> 


We can use applications to examine the variable names in our radar files (e.g., ncdump or RadxPrint). To reduce the application output, you can also pipe information to commands like <code lang="bash">grep</code> or <code lang="bash">head</code>. 
 
<i>Choose one of the ncdump or RadxPrint commands to run below in the next Jupyter cell. If you prefer to run them in a terminal window, remove the "!" symbol, which is just for Jupyter notebooks.</i>

<code lang="bash">!ncdump -h ${BASE_DIR}/data/wind_guided/input/cfrad.*DOW6_PPI.nc </code>

<code lang="bash">!ncdump -h ${BASE_DIR}/data/wind_guided/input/cfrad.*DOW7_PPI.nc </code>

<code lang="bash">!RadxPrint -f ${BASE_DIR}/data/wind_guided/input/cfrad.*DOW6_PPI.nc</code>
 
<code lang="bash">!RadxPrint -f ${BASE_DIR}/data/wind_guided/input/cfrad.*DOW7_PPI.nc </code>


**Hint #1:** you will notice two variables with "Horizontal copolar reflectivity" in the long name. Pick the variable that is present in both files.

**Hint #2:** these files were manually quality controlled to remove bad data and unfold the velocity field, which is why there are so many Doppler velocity fields. **Use VR for the Doppler velocity field.**
</div>


<div class="alert alert-block alert-info"> <b>Task: define the radar variable names in the parameter file based on the output discovered above.</b> 
</div>

**Congrats! You've finished setting up the FRACTL parameter file!**

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import xarray as xr
import matplotlib as mpl
import cartopy.crs as ccrs
from metpy.plots import ctables
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
mpl.rcParams['figure.dpi'] = 300


### 6. Set the SAMURAI parameters

#### 6.1 Set up the center file
One difference between FRACTL and SAMURAI is the requirement of a centerfile, which allows for a moving domain. We will modify and run the Generate_center.pl perl script to generate a time-series of center locations by providing an estimation of the moving speed of the target and the reference location.

| Parameter | Line # | Description |
| --- | --- | --- |
|lat_origin, lon_origin| 13, 14 |Latitude and longitude of reference frame at origin time.|
|u, v| 15, 16 |Storm motion.|
|ymd| 17 |Date of initial time (YYYYMMDD).|
|hr, mn, sec| 18, 19, 20 |Initial hour, minute, second of track.|
|duration| 21 |Duration of track in seconds.|
|origintime| 22 |Time when the frame is at the latitude/longitude at the origin time in seconds from the initial time.|

<div class="alert alert-block alert-info"> <b>Task: open the Generate_center.pl script and modify the parameters for this case.</b> 

The script can be found here: /home/jovyan/lrose-hub/params/intro_samurai/Generate_center.pl
    
1. For lat_origin and lon_origin, use 41.6144, -104.252. In this example, we're using the location of DOW7 as our origin point. 
    
2. For this example, we will use a stationary domain, so set u and v to zero.  

3. The initial time set by ymd, hr, mn, and sec should be set such that the data all occur after the initial time. Here, use 20090605, 22, 04, 41.

4. Duration sets the length of the analysis. Use 18 seconds here. 

5. Origintime is less meaningful in a stationary nest, but for this example use 9. <i>Make a note of the HH:MM:SS time stamp that corresponds to the origin time - we'll need it in the SAMURAI parameter file later.</i>
</div>


#### 6.2 Generate the center file

<div class="alert alert-block alert-info"> <b>Task: run the Perl script to generate the center file and move the center file to the wind_guided input directory.</b> 
    <br>
First, we need to make the script executable so we can run it. Run the following command to change the permissions.
    
<code lang="bash">!chmod u+x ${BASE_DIR}/params/intro_samurai/Generate_center.pl</code>

Generate the .cen file in the input directory using the following command:
<code lang="bash">!${BASE_DIR}/params/intro_samurai/Generate_center.pl</code>
</div>

Generate_center.pl creates the file in the directory containing this notebook. We just need to move it to the input directory, which we can do with the following command. Using either a terminal or the nagivation at the left of the notebook, double check that the file is now in the input directory.

In [ ]:
# uncomment this command to move the new parameter file over
# to avoid overwriting your file later, recomment after you've run it
# !mv ${BASE_DIR}/notebooks/*.cen ${BASE_DIR}/data/intro_samurai/input

#### 6.3 Generate the SAMURAI parameter file
Like FRACTL, SAMURAI uses the <code lang="bash">-print_params</code> flag to generate parameter files with the defaults. We just need to specify the location and name of the parameter file. We have already set up a directory where you can place the parameter files. Feel free to change the name of the file if you prefer. 

<div class="alert alert-block alert-info"> <b>Task: generate the SAMURAI parameter file.</b> 
<br>
Copy the following command into the open cell below and run it!
    
<code lang="bash">!samurai -print_params > ${BASE_DIR}/params/intro_samurai/samurai_params</code>
</div>

<i>We suggest commenting out the command once you've generated the parameter file so you don't accidentally overwrite it in the future.</i>

#### 6.4 Modify the SAMURAI parameter file

Now we will step through and edit the parameter file.

First, we'll start with the parameters associated with the input/output data directories and filenames.

| Parameter | Line # | Description |
| --- | --- | --- |
|data_directory| 171 |The input data directory. It can be a relative path, but an absolute path is more certain.|
|output_directory| 181 |The output directory where the analysis will go.|

<div class="alert alert-block alert-info"> <b>Task: set the data directories.</b> 

Change the data_directory and output_directory parameters to the following.

In the parameter file, change <code lang="bash">data_directory = "/home/jovyan/lrose-hub/data/intro_samurai/input"</code>

In the parameter file, change <code lang="bash">output_directory = "/home/jovyan/lrose-hub/data/intro_samurai/output_sam"</code>
</div>


---
Next, we'll define the cartesian grid.

| Parameter | Line # | Description |
| --- | --- | --- |
|i_min, i_max, i_incr| 254, 260, 266 |Define the minimum and maximum extent and grid spacing in km of the first horizontal dimension (e.g., Cartesian 'x' dimension). For cylindrical analyses (RTZ mode), this is radius.|
|j_min, j_max, j_incr| 272, 278, 284 |Define the minimum and maximum extent and grid spacing of the second horizontal dimension (e.g., Cartesian 'y' dimension in km). For cylindrical analyses (RTZ mode), this is azimuth (degrees).|
|k_min, k_max, k_incr| 290, 296, 302 |Define the minimum and maximum extent and grid spacing in km of the vertical dimension (i.e., 'z' dimension). k_min should usually be 0.|

<div class="alert alert-block alert-info"> <b>Task: set up the SAMURAI grid.</b> 

Using the grid spacing and span that we used to set up FRACTL, fill out these 9 parameters in the SAMURAI parameter file. Since we are creating a Cartesian analysis, i, j, and k correspond to x, y, and z.

</div>


---
SAMURAI requires the user to set the background time. 

| Parameter | Line # | Description |
| --- | --- | --- |
|ref_time| 324 |This is the reference time to which all data will be time-space corrected. It must be a valid time in the centerfile or an error will occur. The time should be the same as the origin time corresponding to the latitude/longitude in the Perl center file script.|

<div class="alert alert-block alert-info"> <b>Task: define the reference time.</b> 

Remember the origin time that we defined in seconds from the center file script? Calculate the timestamp that corresponds to that time in HH:MM:SS.

Change <code lang="bash">ref_time</code> in the parameter file to the origin time.

</div>


---
Next, we'll set some remaining parameters which define the radar variable names, radii of influence, and a basic mask. 

| Parameter | Line # | Description |
| --- | --- | --- |
|radar_dbz| 382 |The name of the reflectivity field in the radar data.|
|radar_vel| 392 |The name of the Doppler velocity field in the radar data.|
|radar_sw| 398 |The name of the spectrum width field in the radar data. This value is used to set the observational error for the Doppler velocity. If you don't have spectrum width, you can point it at another variable (like NCP) or create a simple uniform field.|
|i_reflectivity_roi, j_reflectivity_roi, k_reflectivity_roi| 404, 410, 416 |These variables set the radius of influence for the reflectivity interpolation. They should be similar to the grid increments defined above.|
|mask_reflectivity| 428 |The analysis can be set to missing data where there is no reflectivity. If set to 'None' then no masking is performed. A numerical value will be used as a threshold for the masking, with all data at nodes having less than the given reflectivity value removed. If you are not using a background field then it is generally a good idea to set this to some small number (like -10 dBZ) since the winds are not valid if there is no radar data.|

<div class="alert alert-block alert-info"> <b>Task: fill out the radar-specific parameters</b> 

1. In the parameter file, change the <code lang="bash">radar_dbz</code> and <code lang="bash">radar_vel</code> parameters using the same field names that were used in the FRACTL parameter file. In the parameter file, change <code lang="bash">radar_sw</code> to SW.

2. For the radii of influence (<code lang="bash">i_reflectivity_roi</code>, <code lang="bash">j_reflectivity_roi</code>, <code lang="bash">k_reflectivity_roi</code>), the i and j values can be changed to 0.2 and the k value can be changed to 0.5, similar to the grid spacings.

3. Finally, <code lang="bash">mask_reflectivity</code> can be set to a low value to remove weak echo regions. A value around -25 dBZ is often sufficient.

</div>


### 7. Run SAMURAI

After modifying the parameter file, direct to the parameter file by using the *-params* flags


In [ ]:
!samurai -params ${BASE_DIR}/params/intro_samurai/samurai_params 


### 8. Plot SAMURAI results

#### Environment and packages¶
The packages were already imported when we plotted the FRACTL results.

#### Load the netcdf file

In [ ]:
inDir_s = base_dir+"/data/intro_samurai/output/"
file_s = "samurai_XYZ_analysis.nc"
ds_radar_s = xr.open_dataset(inDir_s+file_s).squeeze()
ds_radar_s.load()


In [ ]:
## Get variables:
da_s_DBZ = ds_radar_s['DBZ']
da_s_U = ds_radar_s['U']
da_s_V = ds_radar_s['V']
da_s_W = ds_radar_s['W']


In [ ]:
## Set NWS reflectivity colorbar:
ref_norm, ref_cmap = ctables.registry.get_with_steps('NWSStormClearReflectivity', -20, 0.5)
plotting_alt=1.5 # altitude at 1.5 km
plotting_var_s = ds_radar_s.DBZ.sel(altitude=plotting_alt)
plotting_var_u_s = ds_radar_s.U.sel(altitude=plotting_alt).data
plotting_var_v_s = ds_radar_s.V.sel(altitude=plotting_alt).data
plotting_lon_s = ds_radar_s.longitude
plotting_lat_s = ds_radar_s.latitude

fig = plt.figure(figsize=(12,12))
ax = plt.axes(projection=ccrs.PlateCarree())
# stamen_terrain = cimgt.Stamen('terrain')
# ax.add_image(stamen_terrain, 8)
       
cf1 = ax.pcolormesh( plotting_lon_s, plotting_lat_s, plotting_var_s
                    , cmap=ref_cmap, norm=ref_norm
                    , alpha=0.8
                    , shading='auto'
                    , transform=ccrs.PlateCarree() 
                    )
step = 4
cf_q = ax.quiver( plotting_lon_s[::step], plotting_lat_s[::step]
                    , plotting_var_u_s[::step, ::step], plotting_var_v_s[::step, ::step]
                    , scale=1000
                    , width=0.004
                    , color='k'
                    , transform=ccrs.PlateCarree() 
                    )

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

cbar_ax = fig.add_axes([0.95, 0.3, 0.02, 0.4])
cbar = fig.colorbar(cf1, cax=cbar_ax, fraction=0.04)
cbar.ax.tick_params(labelsize=14)
cbar.ax.set_title('[dBZ]', fontsize=14, y=-0.1)


### Congratulations! You have successfully completed the Introductory SAMURAI tutorial. There are many scientific parameters to set in SAMURAI, so please check out the Guided Wind and Intermediate SAMURAI tutorials to learn more. Be critical with your own analysis and feel free to reach out to the LROSE team for questions as you analyze your own data.